### Logistic Regression

#### Import Packages

In [16]:
%run ml_helpers.ipynb
%run ml.ipynb

               ID  Domestic  Beat  Year Month  Week  Day  Hour   Watch  PRCP  \
1063071  11540867     False  1511  2018    12    51   19    19   Third  0.00   
1091657  11583025     False  1233  2019     2     5    1    23   Third  0.02   
1279011  11859954     False   812  2019    10    42   14     0   First  0.00   
1275072  11854644     False  2521  2019    10    41    8    13  Second  0.00   
1002341  11458728      True   513  2018     9    39   26    12  Second  0.00   
...           ...       ...   ...   ...   ...   ...  ...   ...     ...   ...   
988701   11440340     False   221  2018     9    36    8    14  Second  0.00   
1172673  11704349     False  1131  2019     5    22   29    20   Third  0.10   
1144434  11663441     False  1531  2019     4    17   22     5   First  0.18   
1112846  11615518      True  2232  2019     3    10    6    19   Third  0.00   
904993   11329403     False   433  2018     5    22   28    19   Third  0.00   

         ...  count_restaurants  count_

Finished one-hot encoding...
Finished standardizing...
Working on: [2016 2015]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
               ID  Domestic  Beat  Year Month  Week  Day  Hour   Watch  PRCP  \
1107473  11607195     False  1212  2019     2     9   26     1   First  0.00   
1213298  11767843     False  1831  2019     7    29   20     0   First  0.00   
1052183  11525742     False  1833  2018    12    49    4    20   Third  0.00   
1007525  11466364     False  2212  2018    10    40    3     7   First  0.00   
1248473  11815075     False  1412  2019     9    36    2    15  Second  0.00   
1040939  11602498     False   132  2018    11    46   18    12  Second  0.10   
1158818  11684032     False   512  2019     5    19   11    19   Third  0.10   
1072896  11554706     False

Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
         Beat  Year  Month   Watch  PRCP  SNOW  TMAX  TMIN  count_l_stops  \
1264551   414  2019      9   Third  0.00   0.0    75    62            0.0   
1094913  2423  2019      2   Third  0.08   0.0    38    33            1.0   
1281050  2013  2019     10   Third  0.00   0.0    53    43            0.0   
1186809  1653  2019      6   Third  0.12   0.0    67    56            0.0   
1143621  1712  2019      4   Third  0.00   0.0    65    41            0.0   
...       ...   ...    ...     ...   ...   ...   ...   ...            ...   
1222777  1512  2019      7   Third  0.00   0.0    76    63            2.0   
1239207  2013  2019      8  Second  0.00   0.0    84    65            0.0   
1141995   332  2019      4  Second  0.50   0.0    65    44            0.0   
1258317   9

Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
Working on: [2016]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
Working on: [2015]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...


In [2]:
import pandas as pd
import geopandas as gpd
import seaborn as sns
import numpy as np
import sklearn as sk
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score
import datetime

#### Read and Process Data

In [9]:
data = pd.read_csv("../intermediate_data/high_crime_labeled.csv")
data = data.drop(columns=["beat", "beat_num", "Crimes", "Arrest", "Domestic", "Serious"])
data = convert_to_categorical(data, ["district", "sector",
                                    "Month",
                                     "Watch", "Beat"])
data.drop(columns = ["Unnamed: 0"], inplace=True)

In [4]:
data.dtypes

district               category
sector                 category
Year                      int64
Month                  category
Watch                  category
Beat                   category
count_l_stops           float64
count_bus_stops         float64
count_metra_stops       float64
count_restaurants       float64
count_bars              float64
count_daycares          float64
count_entertainment     float64
count_businesses        float64
road_distance_ft        float64
TOTAL POPULATION        float64
dist_to_police          float64
dist_to_hospital        float64
SNOW                    float64
TMAX                    float64
TMIN                    float64
high_crime              float64
high_crime_geog_lag     float64
dtype: object

#### Setup ML

In [11]:
models = {
    'LogisticRegression': LogisticRegression(), 
}
grid = {
    'LogisticRegression': [{'penalty': x, 'C': y, 'random_state': 0} 
                           for x in ('l2', 'none') \
                           for y in (0.001, 0.01, 0.1, 1, 10, 100, 1000)]
}

In [14]:
grid

{'LogisticRegression': [{'penalty': 'l2', 'C': 0.001, 'random_state': 0},
  {'penalty': 'l2', 'C': 0.01, 'random_state': 0},
  {'penalty': 'l2', 'C': 0.1, 'random_state': 0},
  {'penalty': 'l2', 'C': 1, 'random_state': 0},
  {'penalty': 'l2', 'C': 10, 'random_state': 0},
  {'penalty': 'l2', 'C': 100, 'random_state': 0},
  {'penalty': 'l2', 'C': 1000, 'random_state': 0},
  {'penalty': 'none', 'C': 0.001, 'random_state': 0},
  {'penalty': 'none', 'C': 0.01, 'random_state': 0},
  {'penalty': 'none', 'C': 0.1, 'random_state': 0},
  {'penalty': 'none', 'C': 1, 'random_state': 0},
  {'penalty': 'none', 'C': 10, 'random_state': 0},
  {'penalty': 'none', 'C': 100, 'random_state': 0},
  {'penalty': 'none', 'C': 1000, 'random_state': 0}]}

In [6]:
data_list = prep_data(data, "high_crime",
                                        2, "Year", ["district", "sector", 
                                                    "Month", 
                                                    "Beat", "Watch"])

      district sector  Year Month   Watch  Beat  count_l_stops  \
29557       17      1  2018     1   First  1713            3.0   
29558       17      1  2018     1  Second  1713            3.0   
29559       17      1  2018     1   Third  1713            3.0   
29560       17      1  2018     2   First  1713            3.0   
29561       17      1  2018     2  Second  1713            3.0   
...        ...    ...   ...   ...     ...   ...            ...   
49256        3      1  2019    11  Second   312            2.0   
49257        3      1  2019    11   Third   312            2.0   
49258        3      1  2019    12   First   312            2.0   
49259        3      1  2019    12  Second   312            2.0   
49260        3      1  2019    12   Third   312            2.0   

       count_bus_stops  count_metra_stops  count_restaurants  ...  \
29557             26.0                0.0               63.0  ...   
29558             26.0                0.0               63.0  ...   


Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
Working on: [2016 2017]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
Working on: [2015 2016]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...


In [15]:
logistic_results = average_grid_searches(LogisticRegression(), "LogisticRegression", 
                          grid, 
                          data_list, "high_crime", 2020)

test year is: 2020
[2016, 2017, 2018, 2019]


In [12]:
logistic_results.to_csv("../final_data/log_reg_results.csv")

In [13]:
logistic_results

,parameters_2019,precision_2019,parameters_2018,precision_2018,parameters_2017,precision_2017,mean
0,"{'penalty': 'l2', 'C': 0.001, 'random_state': 0}",0.764843,"{'penalty': 'l2', 'C': 0.001, 'random_state': 0}",0.782402,"{'penalty': 'l2', 'C': 0.001, 'random_state': 0}",0.755897,0.767714
0,"{'penalty': 'l2', 'C': 0.01, 'random_state': 0}",0.736949,"{'penalty': 'l2', 'C': 0.01, 'random_state': 0}",0.731973,"{'penalty': 'l2', 'C': 0.01, 'random_state': 0}",0.728563,0.732495
0,"{'penalty': 'l2', 'C': 1, 'random_state': 0}",0.723355,"{'penalty': 'l2', 'C': 1, 'random_state': 0}",0.730464,"{'penalty': 'l2', 'C': 1, 'random_state': 0}",0.715122,0.722980
0,"{'penalty': 'none', 'C': 0.001, 'random_state'...",0.719931,"{'penalty': 'none', 'C': 0.001, 'random_state'...",0.719673,"{'penalty': 'none', 'C': 0.001, 'random_state'...",0.715348,0.718317
0,"{'penalty': 'none', 'C': 0.01, 'random_state': 0}",0.719931,"{'penalty': 'none', 'C': 0.01, 'random_state': 0}",0.719673,"{'penalty': 'none', 'C': 0.01, 'random_state': 0}",0.715348,0.718317
0,"{'penalty': 'none', 'C': 0.1, 'random_state': 0}",0.719931,"{'penalty': 'none', 'C': 0.1, 'random_state': 0}",0.719673,"{'penalty': 'none', 'C': 0.1, 'random_state': 0}",0.715348,0.718317
0,"{'penalty': 'none', 'C': 1, 'random_state': 0}",0.719931,"{'penalty': 'none', 'C': 1, 'random_state': 0}",0.719673,"{'penalty': 'none', 'C': 1, 'random_state': 0}",0.715348,0.718317
0,"{'penalty': 'none', 'C': 10, 'random_state': 0}",0.719931,"{'penalty': 'none', 'C': 10, 'random_state': 0}",0.719673,"{'penalty': 'none', 'C': 10, 'random_state': 0}",0.715348,0.718317
0,"{'penalty': 'none', 'C': 100, 'random_state': 0}",0.719931,"{'penalty': 'none', 'C': 100, 'random_state': 0}",0.719673,"{'penalty': 'none', 'C': 100, 'random_state': 0}",0.715348,0.718317
0,"{'penalty': 'none', 'C': 1000, 'random_state': 0}",0.719931,"{'penalty': 'none', 'C': 1000, 'random_state': 0}",0.719673,"{'penalty': 'none', 'C': 1000, 'random_state': 0}",0.715348,0.718317
